# **뉴스 데이터 - 부동산 뉴스기사 키워드 추출을 통한 연도별 이슈 분석**

- 필요한 라이브러리 다운로드

In [81]:
#한글 폰트 다운로드
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf


'sudo'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'sudo'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'rm'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [82]:
#크롤링 user_agent 생성 라이브러리 다운로드
!pip install user_agent

^C


In [ ]:
#한글 형태소 분석기 다운로드
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

������ ������ ã�� �� �����ϴ�.


## **데이터 수집**

- 2018년도부터 2023년 8월까지 '부동산' 키워드 뉴스 기사 크롤링

In [ ]:
from user_agent import generate_user_agent, generate_navigator
user_agent = generate_user_agent()
user_agent

'Mozilla/5.0 (Windows NT 6.3; Win64; x64; Trident/7.0; rv:11.0) like Gecko'

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time
from tqdm.notebook import tqdm


In [ ]:
pd.date_range('2018-01-01', '2023-08-31', freq='M').strftime('%Y.%m.%d')

C:\Users\User\AppData\Local\Temp\ipykernel_16400\2317508191.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pd.date_range('2018-01-01', '2023-08-31', freq='M').strftime('%Y.%m.%d')


Index(['2018.01.31', '2018.02.28', '2018.03.31', '2018.04.30', '2018.05.31',
       '2018.06.30', '2018.07.31', '2018.08.31', '2018.09.30', '2018.10.31',
       '2018.11.30', '2018.12.31', '2019.01.31', '2019.02.28', '2019.03.31',
       '2019.04.30', '2019.05.31', '2019.06.30', '2019.07.31', '2019.08.31',
       '2019.09.30', '2019.10.31', '2019.11.30', '2019.12.31', '2020.01.31',
       '2020.02.29', '2020.03.31', '2020.04.30', '2020.05.31', '2020.06.30',
       '2020.07.31', '2020.08.31', '2020.09.30', '2020.10.31', '2020.11.30',
       '2020.12.31', '2021.01.31', '2021.02.28', '2021.03.31', '2021.04.30',
       '2021.05.31', '2021.06.30', '2021.07.31', '2021.08.31', '2021.09.30',
       '2021.10.31', '2021.11.30', '2021.12.31', '2022.01.31', '2022.02.28',
       '2022.03.31', '2022.04.30', '2022.05.31', '2022.06.30', '2022.07.31',
       '2022.08.31', '2022.09.30', '2022.10.31', '2022.11.30', '2022.12.31',
       '2023.01.31', '2023.02.28', '2023.03.31', '2023.04.30', '2023.05.31',

In [ ]:
#pd.date_range('2018-01-01', '2023-08-31', freq='MS').strftime('%Y.%m.%d') => freq='MS' 각 월의 첫번째날 / freq='M' 각 월의 마지막날   

first_day = pd.date_range('2018-01-01', '2024-10-31', freq='MS').strftime('%Y.%m.%d')
last_day = pd.date_range('2018-01-01', '2024-10-31', freq='M').strftime('%Y.%m.%d')

# Zip을 통해서 리스트 안에 (first_day, last_day)가 들어가게 만들어 준다.
date_list = list(zip(first_day, last_day))

C:\Users\User\AppData\Local\Temp\ipykernel_16400\975868755.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  last_day = pd.date_range('2018-01-01', '2024-10-31', freq='M').strftime('%Y.%m.%d')


In [ ]:
date_list

[('2018.01.01', '2018.01.31'),
 ('2018.02.01', '2018.02.28'),
 ('2018.03.01', '2018.03.31'),
 ('2018.04.01', '2018.04.30'),
 ('2018.05.01', '2018.05.31'),
 ('2018.06.01', '2018.06.30'),
 ('2018.07.01', '2018.07.31'),
 ('2018.08.01', '2018.08.31'),
 ('2018.09.01', '2018.09.30'),
 ('2018.10.01', '2018.10.31'),
 ('2018.11.01', '2018.11.30'),
 ('2018.12.01', '2018.12.31'),
 ('2019.01.01', '2019.01.31'),
 ('2019.02.01', '2019.02.28'),
 ('2019.03.01', '2019.03.31'),
 ('2019.04.01', '2019.04.30'),
 ('2019.05.01', '2019.05.31'),
 ('2019.06.01', '2019.06.30'),
 ('2019.07.01', '2019.07.31'),
 ('2019.08.01', '2019.08.31'),
 ('2019.09.01', '2019.09.30'),
 ('2019.10.01', '2019.10.31'),
 ('2019.11.01', '2019.11.30'),
 ('2019.12.01', '2019.12.31'),
 ('2020.01.01', '2020.01.31'),
 ('2020.02.01', '2020.02.29'),
 ('2020.03.01', '2020.03.31'),
 ('2020.04.01', '2020.04.30'),
 ('2020.05.01', '2020.05.31'),
 ('2020.06.01', '2020.06.30'),
 ('2020.07.01', '2020.07.31'),
 ('2020.08.01', '2020.08.31'),
 ('2020.

In [ ]:
url = 'https://search.naver.com/search.naver?where=news&query=%EB%B6%80%EB%8F%99%EC%82%B0&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2023.01.01&de=2023.01.31&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20230101to20230131&is_sug_officeid=0&office_category=0&service_area=1'

In [ ]:
user_agent = generate_user_agent()
headers = {'user-Agent':user_agent}

res = requests.get(url, headers=headers)
soup = bs(res.text, 'html.parser')

In [ ]:
# 뉴스의 타이틀을 불러오기 위한 것
soup.find_all('a', class_='news_tit')[0].text

'전국 미분양 결국 레드라인 넘었다…7만 가구 육박 [부동산360]'

In [ ]:
[i.text for i in soup.find_all('a', class_='news_tit')]

['전국 미분양 결국 레드라인 넘었다…7만 가구 육박 [부동산360]',
 '부동산원-부동산R114, 입주물량 공개...올해 44만3000가구 예정',
 '부동산 대출 규제 더 풀린다… 1주택자 LTV 추가 확대 추진',
 '부동산PF 시장에도 위기감 계속‥정부, 부실 PF 연착륙 유도',
 '[단독] NH투자증권, 부동산PF 운용사 구상권 청구 소송',
 '주담대 원금 상환 유예해주고…부동산 대출 규제는 더 푼다',
 '금융위,올해 최대리스크는 부동산PF..1조원 펀드 조성',
 "최대 뇌관 'PF 대책' 내놓은 금융위...부동산 경착륙 막을까",
 "'500억대 부동산 부자' 송중기도 받는다…다문화가정 혜택은",
 "원희룡 장관과 오세훈 시장의 '부동산 말 한마디' [김진수의 부동산 인..."]

In [ ]:
[i['href'] for i in soup.find_all('a', class_='news_tit')]

['http://news.heraldcorp.com/view.php?ud=20230131000515',
 'https://www.news1.kr/articles/4938499',
 'http://www.segye.com/content/html/2023/01/30/20230130516606.html?OutUrl=naver',
 'https://imnews.imbc.com/replay/2023/nwdesk/article/6450602_36199.html',
 'https://biz.sbs.co.kr/article_hub/20000100868?division=NAVER',
 'https://www.nocutnews.co.kr/news/5886816',
 'http://www.fnnews.com/news/202301301406404010',
 'http://www.ohmynews.com/NWS_Web/View/at_pg.aspx?CNTN_CD=A0002898074&CMPT_CD=P0010&utm_source=naver&utm_medium=newsearch&utm_campaign=naver_news',
 'https://www.joongang.co.kr/article/25137213',
 'https://www.hankyung.com/realestate/article/202301313595i']

In [ ]:
[i.text for i in soup.find_all('a', class_='info press')]

['헤럴드경제언론사 선정',
 '뉴스1',
 '세계일보언론사 선정',
 'MBC언론사 선정',
 'SBS Biz언론사 선정',
 '노컷뉴스언론사 선정',
 '파이낸셜뉴스언론사 선정',
 '오마이뉴스언론사 선정',
 '중앙일보언론사 선정',
 '한국경제언론사 선정']

In [ ]:
for dt in tqdm(date_list):
    dt1 = [dt[0].replace('.',''), dt[1].replace('.','')]
    print(dt, dt1)

  0%|          | 0/82 [00:00<?, ?it/s]

('2018.01.01', '2018.01.31') ['20180101', '20180131']
('2018.02.01', '2018.02.28') ['20180201', '20180228']
('2018.03.01', '2018.03.31') ['20180301', '20180331']
('2018.04.01', '2018.04.30') ['20180401', '20180430']
('2018.05.01', '2018.05.31') ['20180501', '20180531']
('2018.06.01', '2018.06.30') ['20180601', '20180630']
('2018.07.01', '2018.07.31') ['20180701', '20180731']
('2018.08.01', '2018.08.31') ['20180801', '20180831']
('2018.09.01', '2018.09.30') ['20180901', '20180930']
('2018.10.01', '2018.10.31') ['20181001', '20181031']
('2018.11.01', '2018.11.30') ['20181101', '20181130']
('2018.12.01', '2018.12.31') ['20181201', '20181231']
('2019.01.01', '2019.01.31') ['20190101', '20190131']
('2019.02.01', '2019.02.28') ['20190201', '20190228']
('2019.03.01', '2019.03.31') ['20190301', '20190331']
('2019.04.01', '2019.04.30') ['20190401', '20190430']
('2019.05.01', '2019.05.31') ['20190501', '20190531']
('2019.06.01', '2019.06.30') ['20190601', '20190630']
('2019.07.01', '2019.07.31')

In [ ]:
keyword = '부동산'

data = pd.DataFrame()
for dt in tqdm(date_list):
    dt1 = [dt[0].replace('.',''), dt[1].replace('.','')]

    for num in range(1, 101, 10):
        url = f'https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={dt[0]}&de={dt[1]}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{dt1[0]}to{dt1[1]}&is_sug_officeid=0&office_category=0&service_area=1&start={num}'
        
        user_agent = generate_user_agent()
        headers = {'user-Agent':user_agent}

        res = requests.get(url, headers=headers)
        time.sleep(random.random())
        soup = bs(res.text, 'html.parser')

        # 뉴스 타이틀
        title = [i.text for i in soup.find_all('a', class_='news_tit')]
        # 뉴스 링크
        link = [i['href'] for i in soup.find_all('a', class_='news_tit')]
        # 뉴스 언론사
        press = [i.text for i in soup.find_all('a', class_='info press')]

        temp = pd.DataFrame({'title':title, 'link':link, 'press':press, 'date_ym':dt1[0][:6]})
        data = pd.concat([data,temp])

  0%|          | 0/82 [00:00<?, ?it/s]

In [ ]:
data.to_excel('C:\\Users\\User\\Desktop\\패스트 캠퍼스 데이터 마스터\\파이썬\\Part3) 파이썬 데이터 분석 실습\Data\\실습7_ 직접 크롤링하기 - 부동산 뉴스기사 키워드 추출을 통한 연도별 이슈 분석\\news_data.xlsx')

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\User\AppData\Local\Temp\ipykernel_16400\2251310781.py:1: SyntaxWarning: invalid escape sequence '\D'
  data.to_excel('C:\\Users\\User\\Desktop\\패스트 캠퍼스 데이터 마스터\\파이썬\\Part3) 파이썬 데이터 분석 실습\Data\\실습7_ 직접 크롤링하기 - 부동산 뉴스기사 키워드 추출을 통한 연도별 이슈 분석\\news_data.xlsx')


In [ ]:
data = pd.read_excel('C:\\Users\\User\\Desktop\\패스트 캠퍼스 데이터 마스터\\파이썬\\Part3) 파이썬 데이터 분석 실습\Data\\실습7_ 직접 크롤링하기 - 부동산 뉴스기사 키워드 추출을 통한 연도별 이슈 분석\\news_data.xlsx')
data

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\User\AppData\Local\Temp\ipykernel_16400\3035977554.py:1: SyntaxWarning: invalid escape sequence '\D'
  data = pd.read_excel('C:\\Users\\User\\Desktop\\패스트 캠퍼스 데이터 마스터\\파이썬\\Part3) 파이썬 데이터 분석 실습\Data\\실습7_ 직접 크롤링하기 - 부동산 뉴스기사 키워드 추출을 통한 연도별 이슈 분석\\news_data.xlsx')


,Unnamed: 0,title,link,press,date_ym
0,0,부자들 60% “부동산 규제에도 집 안 판다”,http://www.hankookilbo.com/v/26932a67d1b242f99...,한국일보언론사 선정,201801
1,1,"""파느니 물려준다"" 지난해 부동산 증여 28만2천건 역대 최대",http://app.yonhapnews.co.kr/YNA/Basic/SNS/r.as...,연합뉴스,201801
2,2,부동산 중개사이트 허위매물 제재한다,http://news.heraldcorp.com/view.php?ud=2018013...,헤럴드경제,201801
3,3,"김현미 장관 ""부동산 침체 지역, '위축지역' 지정 검토""",http://www.newsis.com/view/?id=NISX20180130_00...,뉴시스언론사 선정,201801
4,4,"KB운용, 글로벌 대체투자 강화…해외부동산 전담본부 신설",http://news.mt.co.kr/mtview.php?no=20180131085...,머니투데이,201801
...,...,...,...,...,...
8195,5,강남서 '지분 쪼개기' 성행… 그린벨트 해제 앞두고 시끌,https://www.moneys.co.kr/article/2024103017085...,머니S언론사 선정,202410
8196,6,대출금리↑ 예금금리↓…지역부동산 발목잡는 ‘관치금융’,http://www.kookje.co.kr/news2011/asp/newsbody....,국제신문언론사 선정,202410
8197,7,서울 아파트값 주춤하는데...강남·재건축 단지는 신고가,http://www.segye.com/newsView/20241031519104?O...,세계일보언론사 선정,202410
8198,8,"""의사 아닌 부동산중개인이 털 이식수술?""...부작용 끔찍, 스스로 목숨 ...",https://kormedi.com/?p=1732336,코메디닷컴언론사 선정,202410


In [ ]:
data.drop('Unnamed: 0',axis=1, inplace=True)

In [ ]:
data['date_ym'] = data['date_ym'].astype(str)

## **질문 만들기**

- 연도별로 부동산 전망에 대한 뉴스 키워드는 어떻게 변화했는가?
    - 워드클라우드로 시각화

## **데이터 전처리**

- 필요 없는 컬럼 제거 및 데이터 타입 변경

In [ ]:
data = pd.read_excel('C:\\Users\\User\\Desktop\\패스트 캠퍼스 데이터 마스터\\파이썬\\Part3) 파이썬 데이터 분석 실습\Data\\실습7_ 직접 크롤링하기 - 부동산 뉴스기사 키워드 추출을 통한 연도별 이슈 분석\\news_data.xlsx')
data.head()

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\User\AppData\Local\Temp\ipykernel_16400\1355907361.py:1: SyntaxWarning: invalid escape sequence '\D'
  data = pd.read_excel('C:\\Users\\User\\Desktop\\패스트 캠퍼스 데이터 마스터\\파이썬\\Part3) 파이썬 데이터 분석 실습\Data\\실습7_ 직접 크롤링하기 - 부동산 뉴스기사 키워드 추출을 통한 연도별 이슈 분석\\news_data.xlsx')


,Unnamed: 0,title,link,press,date_ym
0,0,부자들 60% “부동산 규제에도 집 안 판다”,http://www.hankookilbo.com/v/26932a67d1b242f99...,한국일보언론사 선정,201801
1,1,"""파느니 물려준다"" 지난해 부동산 증여 28만2천건 역대 최대",http://app.yonhapnews.co.kr/YNA/Basic/SNS/r.as...,연합뉴스,201801
2,2,부동산 중개사이트 허위매물 제재한다,http://news.heraldcorp.com/view.php?ud=2018013...,헤럴드경제,201801
3,3,"김현미 장관 ""부동산 침체 지역, '위축지역' 지정 검토""",http://www.newsis.com/view/?id=NISX20180130_00...,뉴시스언론사 선정,201801
4,4,"KB운용, 글로벌 대체투자 강화…해외부동산 전담본부 신설",http://news.mt.co.kr/mtview.php?no=20180131085...,머니투데이,201801


In [ ]:
data.drop('Unnamed: 0',axis=1, inplace=True)

In [ ]:
data['date_ym'] = data['date_ym'].astype(str)

In [ ]:
data = data[['title', 'date_ym']]

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8200 entries, 0 to 8199
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    8200 non-null   object
 1   date_ym  8200 non-null   object
dtypes: object(2)
memory usage: 128.3+ KB


,title,date_ym
0,부자들 60% “부동산 규제에도 집 안 판다”,201801
1,"""파느니 물려준다"" 지난해 부동산 증여 28만2천건 역대 최대",201801
2,부동산 중개사이트 허위매물 제재한다,201801
3,"김현미 장관 ""부동산 침체 지역, '위축지역' 지정 검토""",201801
4,"KB운용, 글로벌 대체투자 강화…해외부동산 전담본부 신설",201801


In [ ]:
%pip install mecab-python3
%pip install konlpy

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


- 형태소 분석

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()

Exception: The MeCab dictionary does not exist at "/usr/local/lib/mecab/dic/mecab-ko-dic". Is the dictionary correctly installed?
You can also try entering the dictionary path when initializing the Mecab class: "Mecab('/some/dic/path')"

In [ ]:
# lambda x: [i[0] for i in mecab.pos(x) if i[i] in ('NNG')]  i의 두번째에NNG가 있는 데이터들의 첫번째 데이터를 가져오는 것

data['NNG'] = data['title'].apply(lambda x: [i[0] for i in mecab.pos(x) if i[i] in ('NNG')])

NameError: name 'mecab' is not defined

In [ ]:
data['year'] = data['date_ym'].apply(lambda x: int(x[:4]))
data.head()

In [ ]:
from collections import Counter 

In [ ]:
# 각 연도에 해당하는 형태소를 각각 연도의 리스트에 모으는것

nng_2018 = [j for i in data.query('year == 2018')['NNG'] for j in i if j not in ['부동산','전망','시장','집값','투자'] and len(j) > 1]
nng_2019 = [j for i in data.query('year == 2019')['NNG'] for j in i if j not in ['부동산','전망','시장','집값','투자'] and len(j) > 1]
nng_2020 = [j for i in data.query('year == 2020')['NNG'] for j in i if j not in ['부동산','전망','시장','집값','투자'] and len(j) > 1]
nng_2021 = [j for i in data.query('year == 2021')['NNG'] for j in i if j not in ['부동산','전망','시장','집값','투자'] and len(j) > 1]
nng_2022 = [j for i in data.query('year == 2022')['NNG'] for j in i if j not in ['부동산','전망','시장','집값','투자'] and len(j) > 1]
nng_2023 = [j for i in data.query('year == 2023')['NNG'] for j in i if j not in ['부동산','전망','시장','집값','투자'] and len(j) > 1]
nng_2024 = [j for i in data.query('year == 2024')['NNG'] for j in i if j not in ['부동산','전망','시장','집값','투자'] and len(j) > 1]


# 각 연도에 해당하는 형태소를 각각 연도의 리스트에 모은 거를 카운트 해주며 most_common은 가장 많이 나온 단어를 추출 해주는것 현재 100이니 가장 많이 나온 단어 100개 저장
nng_2018_dict = dict(Counter(nng_2018).most_common(100))
nng_2019_dict = dict(Counter(nng_2019).most_common(100))
nng_2020_dict = dict(Counter(nng_2020).most_common(100))
nng_2021_dict = dict(Counter(nng_2021).most_common(100))
nng_2022_dict = dict(Counter(nng_2022).most_common(100))
nng_2023_dict = dict(Counter(nng_2023).most_common(100))
nng_2024_dict = dict(Counter(nng_2024).most_common(100))

In [83]:
!pip install wordcloud

ERROR: Could not install packages due to an OSError: [WinError 2] 지정된 파일을 찾을 수 없습니다: 'C:\\Python312\\Scripts\\wordcloud_cli.exe' -> 'C:\\Python312\\Scripts\\wordcloud_cli.exe.deleteme'



## **분석**

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothic')

from wordcloud import WordCloud

path = '/user/share/fonts/truetype/nanum/NanumGothic.ttf'
wc = WordCloud(font_path = path,
               background_color='white',
               width=1000,
               height=1000,
               max_font_size=300)

dict_list = [nng_2018_dict, nng_2019_dict, nng_2020_dict, nng_2021_dict, nng_2022_dict, nng_2023_dict, nng_2024_dict]
title_list = [i for i in range(2018, 2025)]

fig = plt.figure(figsize=(25,20))
for i in range(len(dict_list)):
    wc.generate_from_frequencies(dict_list[i]) #워드클라우드 생성

    # fig.add_subplot(세로, 가로) 한번에 여러 표를 보이게 하기 위해서 하는것 
    ax = fig.add_subplot(2,3,i+1)
    ax.imshow(wc, interpolation='bilinear')
    ax.set_xlabel(f'{title_list[i]}') #그래프 제목 출력
    ax.set_xticks([]), ax.set_yticks([]) #x축, y축을 없앰
    plt.imshow(wc, interpolation='bilinear')
fig.suptitle('부동산 뉴스 제목 키워드')
fig.tight_layout()
plt.show()